# Decision-focused Learning

## Prediction and Optimization in the Wild

**Real world problems typically rely on _estimated parameters_**

E.g. travel times, demands, item weights/costs...

<center><img src="assets/traffic.jpg" width="70%"/></center>

**However, sometimes we have access to _a bit more information_**

## Prediction and Optimization in the Wild

**Take _traffic-dependent travel times_ as an example**

If we know the _time of the day_ we can probably estimate them better

<center><img src="assets/traffic.jpg" width="70%"/></center>

**Let's see how these problems are typically addressed**

## Predict, then Optimize

**First, we _train an estimator_ for the problem parameters:**

$$
\text{argmin}_{\omega} \left\{L(y, \hat{y}) \mid y = f(\hat{x}; \omega) \right\}
$$
* $L$ is the loss function
* $f$ is the ML model with parameter 
* $\hat{x}$, $\hat{y}$ are the training set input/output

**In our example:**

* $x$ would be the time of the day
* $y$ would be a vector of travel times
* $L$ may be a classical MSE loss
* $f$ may be a linear regressor or neural network

## Predict, then Optimize

**Then, we solve the optimization problem with the estimated parameters**

$$
z^*(y) = \text{argmin}_z \left\{ c(z, y) \mid z \in F(y) \right\}
$$
* $z$ is the vector of variables of the optimization problem
* $c$ is the cost function
* $F$ is the feasible space
* In general, both $c$ and $F$ may depend on the estimated parameters

**In our example**

* $z$ may represent routing decisions
* $c$ may be the total travel time
* $F$ may encode a deadline constraint

## Predict, then Optimize

**This approach is sometimes referred to as "Predict, then Optimize"**

It is simple and it makes intuitively sense

* The more accurate we are, the better we will estimate the parameters
* ...And in turn we should get better optimization results

**Let $L^*(\hat{y})$ be the best possible loss value, for any ML model**

* For any reasonable loss function, _better training leads to better predictions_

$$
y \xrightarrow[L(y, \hat{y}) \rightarrow L^*(\hat{y})]{} y^*
$$

* ...And therefore, eventually we are _guaranteed to find the best solution_

$$
z^*(y) \xrightarrow[L(y, \hat{y}) \rightarrow L^*(\hat{y})]{} z^*(\hat{y})
$$

## "Predict, then Optimize": Limitations

**However, things are not really that simple!**

> **Why is that the case?**

The relation:

$$
z^*(y) \xrightarrow[L(y, \hat{y}) \rightarrow L^*(\hat{y})]{} z^*(\hat{y})
$$

...Holds only _asymptotically_

* In practice, our model may not be capable of reaching mimimum loss
* ...And this is even more true for unseen example

In this situation, it is unclear _how imperfect predictions impact the cost_

## "Predict, then Optimize": Limitations

**Say we want to move from location A to B, using one of two routes**

Based on the time of the day (x-axis)

<center class='smalltext'>
<img src="assets/spo_example_1.png" width="45%"/>
Image from <a href="https://arxiv.org/abs/1710.08005">"Smart Predict, then Optimize"</a>
</center>

...The travel time changes (y-axis)

## "Predict, then Optimize": Limitations

**We need to pick the best route**

<center class='smalltext'>
<img src="assets/spo_example_1.png" width="45%"/>
Image from <a href="https://arxiv.org/abs/1710.08005">"Smart Predict, then Optimize"</a>
</center>

* The dashed line shows the input value that causes the optimal choice to switch

## "Predict, then Optimize": Limitations

**If we train an optimal Linear Regression, we get these estimates**

<center><img src="assets/spo_example_2.png" width="40%"/></center>

* The estimator is most accurate possible
* ...But we get the switching point _wrong_!

## "Predict, then Optimize": Limitations

**By contrast, consider this second estimator**

<center><img src="assets/spo_example_3.png" width="40%"/></center>

* The accuracy is awful
* ...But we get the switching point _right_!

## Decision Focused Learning

**Addressing these issues is the goal of _decision focused learning_**

* The general idea is to _account for the optimization problem_ during training
* ...And the "holy grail" of the DFL is solving:

$$
\text{argmin}_{\omega} \left\{ \sum_{i=1}^m c(z^*(y_i), \hat{y}_i) \mid y = f(\hat{x}, \omega) \right\}
$$

* The field was kicked off by [this paper](https://arxiv.org/abs/1703.04529)
* ...And many other have followed

## The Main Challenge

**We are now ready to tackled decision-focused learning**

Let's start from the "holy grail" problem:

$$
\text{argmin}_{\omega} \left\{ \sum_{i=1}^m c(z^*(y_i), \hat{y}_i) \mid y = f(\hat{x}, \omega) \right\}
$$

Unfortunately, the $\text{argmin}$ used to define $z^*(y_i)$ is _non-differentiable_

* A small change in the prediction vector $y_i$
* ...May cause a large/discrete change in the optimal solution $z_i$

This is certainly the case for our example problem

> **How are we going to deal with this?**

# Reinforcement Learning for DFL

Reinforcement Learning (RL) researchers have been working for years on developing methods theat deal with non-differentiable rewards.

**What if the action of the agent is the predicted parameter $\hat{y}_i$ and the reward is the cost of the solution $c(z^*(y), \hat{y})$?**

In this way, we are solving the DFL problem and in principle we can use any RL algorithm.

## Weighted Set Multi-cover with stochastic coverage requirements

Now we introduce the problem we will investigate. We consider a simplified production planning problem, modeled as a Set Multi-cover Problem with stochastic coverage requirements. Given a universe $N$ containing $n$ elements and a collection of sets over $N$, the Set Multi-cover Problem requires finding a minimum size sub-collection of the sets such that coverage requirements for each element are satisfied. The sets may represent products that need to be manufactured together, while the coverage requirements represent product demands.

We consider a version of the problem where sets have non-uniform manufacturing costs, and where the demands are
stochastic and unknown at production time. Unmet demands can then be satisfied by buying additional items, but at a
higher cost. The requirements are generated according to a Poisson distribution, and we assume the existence of a linear
relationships between an observable variable $o ∈ \mathbb{R}$ and the Poisson rates $\lambda$ for each product.

\begin{align}
        \min & \sum_{j \in J} c_j z_j +  \sum_{i \in I} p_{i} s_{i} \label{stocmodel:costfun} \\
             & \sum_{j \in J} a_{i, j} z_j \geq d_{i} (1 - w_{i}) \label{stocmodel:demands} \quad & \forall i \in I \\ 
             & w_{i} = 1 \implies s_{i}  \geq d_{i} - \sum_{j \in J} a_{i, j} z_j \quad & \forall i \in I \label{stocmodel:indicatorconstr} \\
             & z_j \geq 0 \\
             & w_{i} \in \left[ 0, 1 \right] \\
             & s_{i} \geq 0 \\
             & z, w \in \mathbb{Z}
\end{align}

## Dataset generation

First of all, we need to generate the problem instances.

In [1]:
%load_ext autoreload
%autoreload 2
import os
# FIXME: bad code here...
# Run this cell only once to set the execution path in the main folder
os.chdir('..')

In [2]:
import numpy as np
from usecases.wsmc.generate_instances import generate_training_and_test_sets

In [3]:
# Min possible value for the Poisson rates
MIN_LMBD = 1
# Max possible value for the Poisson rates
MAX_LMBD = 10
# Number of products (elements)
NUM_PRODS = 5
# Number of sets (molds)
NUM_SETS = 25
# Density of the availability matrix
DENSITY = 0.02
# Number of instances to generate
NUM_INSTANCES = 1000
# Seed to ensure reproducible results
SEED = 0
DATA_PATH = os.path.join('data',
                         'wsmc',
                         f'{NUM_PRODS}x{NUM_SETS}',
                         'linear',
                         f'{NUM_INSTANCES}-instances',
                         f'seed-{SEED}')

# Set the random seed to ensure reproducibility
np.random.seed(SEED)

In [4]:
# Generate training and test set in the specified directory
generate_training_and_test_sets(data_path=DATA_PATH,
                                num_instances=NUM_INSTANCES,
                                num_sets=NUM_SETS,
                                num_prods=NUM_PRODS,
                                density=DENSITY,
                                min_lmbd=MIN_LMBD,
                                max_lmbd=MAX_LMBD)

[MinSetCover] - True density: 0.264


Saving instances: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.71it/s]


We can find the saved instances in the `DATA_PATH` folder.

## A2C as RL algorithm

As a baseline approached, we have employed the A2C algorithm from the `garage` library.

In [5]:
from garage.tf.baselines import ContinuousMLPBaseline
from garage.sampler import LocalSampler
from garage.tf.policies import GaussianMLPPolicy
from garage.tf.policies import Policy
from garage.experiment import SnapshotConfig

from helpers.garage_utility import CustomTFTrainer, CustomEnv, my_wrap_experiment
from usecases.wsmc.generate_instances import MinSetCoverEnv
from rl.algos import VPG
from helpers.garage_utility import CustomTFTrainer, CustomEnv, my_wrap_experiment
from helpers.utility import renamed_load

import yaml
import os
import tensorflow as tf

c:\users\hp\documents\github\rl-for-dfl\venv\lib\site-packages\garage\experiment\deterministic.py:37: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  'Enabeling deterministic mode in PyTorch can have a performance '


In [6]:
def train(ctxt: SnapshotConfig = None,
          num_epochs: int = 100,
          batch_size: int = 100,
          env: MinSetCoverEnv = None):
    """
    :param ctxt: garage.experiment.SnapshotConfig: The snapshot configuration used by Trainer to create the
                                                   snapshotter.
    :param num_epochs: int; number of training epochs.
    :param batch_size: int; batch size.
    :param env: usecases.setcover.generate_instances.MinSetCoverEnv; the Minimum Set Cover environment instance.
    :return:
    """

    # Check that the env is not None
    assert env is not None

    # A trainer provides a default TensorFlow session using python context
    with CustomTFTrainer(snapshot_config=ctxt) as trainer:

        # Garage wrapping of a gym environment
        env = CustomEnv(env, max_episode_length=1)

        # A policy represented by a Gaussian distribution which is parameterized by a multilayer perceptron (MLP)
        policy = GaussianMLPPolicy(env.spec)
        obs, _ = env.reset()

        # A value function using a MLP network.
        baseline = ContinuousMLPBaseline(env_spec=env.spec)

        # It's called the "Local" sampler because it runs everything in the same process and thread as where
        # it was called from.
        sampler = LocalSampler(agents=policy,
                               envs=env,
                               max_episode_length=1,
                               is_tf_worker=True)

        # Vanilla Policy Gradient
        algo = VPG(env_spec=env.spec,
                   baseline=baseline,
                   policy=policy,
                   sampler=sampler,
                   discount=0,
                   optimizer_args=dict(learning_rate=0.001, ),
                   center_adv=True)

        trainer.setup(algo, env)
        trainer.train(n_epochs=num_epochs, batch_size=batch_size, plot=False)

In [7]:
# Set the hyper parameters
BATCH_SIZE = 100
EPOCHS = 5000

SAVEPATH = os.path.join('models',
                        f'{NUM_PRODS}x{NUM_SETS}',
                        f'{NUM_INSTANCES}-instances',
                        f'seed-{SEED}')

# Create the environment
env = MinSetCoverEnv(num_prods=NUM_PRODS,
                     num_sets=NUM_SETS,
                     instances_filepath=DATA_PATH,
                     seed=SEED)

# Create the saving directory if it does not exist
if not os.path.exists(SAVEPATH):
    os.makedirs(SAVEPATH)

# Save the configuration params
config_params = {'batch_size': BATCH_SIZE, 'epochs': EPOCHS}
with open(os.path.join(SAVEPATH, 'config.yaml'), 'w') as file:
    yaml.dump(config_params, file)

# Train and test the RL algo
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
run = my_wrap_experiment(train,
                         logging_dir=SAVEPATH,
                         snapshot_mode='gap_overwrite',
                         snapshot_gap=EPOCHS // 10,
                         # FIXME: archive_launch_repo=True is not supported
                         archive_launch_repo=False)
run(num_epochs=EPOCHS, batch_size=BATCH_SIZE, env=env)

Gym environment - Loading instances: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 69.50it/s]
c:\users\hp\documents\github\rl-for-dfl\venv\lib\site-packages\keras\legacy_tf_layers\core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


2023-01-12 12:13:19 | [train] Logging to models\5x25\1000-instances\seed-0
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


c:\users\hp\documents\github\rl-for-dfl\venv\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
2023-01-12 12:13:22 | [train] Obtaining samples...
2023-01-12 12:13:24 | [train] epoch #0 | Optimizing policy...
2023-01-12 12:13:24 | [train] epoch #0 | Computing loss before
2023-01-12 12:13:24 | [train] epoch #0 | Computing KL before
2023-01-12 12:13:24 | [train] epoch #0 | Optimizing
Optimizing minibatches
2023-01-12 12:13:25 | [train] epoch #0 | Computing KL after
2023-01-12 12:13:25 | [train] epoch #0 | Computing loss after
2023-01-12 12:13:25 | [train] epoch #0 | Fitting baseline...
2023-01-12 12:13:25 | [train] epoch #0 | Saving snapshot...
0.6628811359405518
2023-01-12 12:13:26 | [train] epoch #0 | Saved
2023-01-12 12:13:26 | [train] epoch #0 | Time 4.43 s
2023-01-12 12:13:26 | [train] epoch #0 | EpochTime 4.43 s
---------------------------------------  -----------------
ContinuousMLPBaseline/ExplainedVariance       -1.02583e-05
ContinuousMLPBaseline/LossAfter                1.02067e+08
Con

2023-01-12 12:13:36 | [train] epoch #4 | Optimizing policy...
2023-01-12 12:13:36 | [train] epoch #4 | Computing loss before
2023-01-12 12:13:36 | [train] epoch #4 | Computing KL before
2023-01-12 12:13:36 | [train] epoch #4 | Optimizing
Optimizing minibatches
2023-01-12 12:13:36 | [train] epoch #4 | Computing KL after
2023-01-12 12:13:36 | [train] epoch #4 | Computing loss after
2023-01-12 12:13:36 | [train] epoch #4 | Fitting baseline...
2023-01-12 12:13:36 | [train] epoch #4 | Saving snapshot...
0.0
2023-01-12 12:13:36 | [train] epoch #4 | Saved
2023-01-12 12:13:36 | [train] epoch #4 | Time 14.56 s
2023-01-12 12:13:36 | [train] epoch #4 | EpochTime 2.58 s
---------------------------------------  ----------------
ContinuousMLPBaseline/ExplainedVariance       0.56536
ContinuousMLPBaseline/LossAfter               8.38802e+07
ContinuousMLPBaseline/LossBefore              8.39625e+07
ContinuousMLPBaseline/dLoss               82304
Evaluation/AverageDiscountedReturn       -15030.2
Evaluat

2023-01-12 12:13:46 | [train] epoch #8 | Optimizing policy...
2023-01-12 12:13:46 | [train] epoch #8 | Computing loss before
2023-01-12 12:13:46 | [train] epoch #8 | Computing KL before
2023-01-12 12:13:46 | [train] epoch #8 | Optimizing
Optimizing minibatches
2023-01-12 12:13:46 | [train] epoch #8 | Computing KL after
2023-01-12 12:13:46 | [train] epoch #8 | Computing loss after
2023-01-12 12:13:46 | [train] epoch #8 | Fitting baseline...
2023-01-12 12:13:46 | [train] epoch #8 | Saving snapshot...
0.0
2023-01-12 12:13:46 | [train] epoch #8 | Saved
2023-01-12 12:13:46 | [train] epoch #8 | Time 24.57 s
2023-01-12 12:13:46 | [train] epoch #8 | EpochTime 2.14 s
---------------------------------------  ----------------
ContinuousMLPBaseline/ExplainedVariance       0.556266
ContinuousMLPBaseline/LossAfter               8.58754e+07
ContinuousMLPBaseline/LossBefore              9.10435e+07
ContinuousMLPBaseline/dLoss                   5.16808e+06
Evaluation/AverageDiscountedReturn       -1313

2023-01-12 12:13:57 | [train] epoch #12 | Optimizing policy...
2023-01-12 12:13:57 | [train] epoch #12 | Computing loss before
2023-01-12 12:13:57 | [train] epoch #12 | Computing KL before
2023-01-12 12:13:57 | [train] epoch #12 | Optimizing
Optimizing minibatches
2023-01-12 12:13:57 | [train] epoch #12 | Computing KL after
2023-01-12 12:13:57 | [train] epoch #12 | Computing loss after
2023-01-12 12:13:57 | [train] epoch #12 | Fitting baseline...
2023-01-12 12:13:57 | [train] epoch #12 | Saving snapshot...
0.0
2023-01-12 12:13:57 | [train] epoch #12 | Saved
2023-01-12 12:13:57 | [train] epoch #12 | Time 35.20 s
2023-01-12 12:13:57 | [train] epoch #12 | EpochTime 2.08 s
---------------------------------------  ----------------
ContinuousMLPBaseline/ExplainedVariance       0.524346
ContinuousMLPBaseline/LossAfter               5.86085e+07
ContinuousMLPBaseline/LossBefore              6.996e+07
ContinuousMLPBaseline/dLoss                   1.13515e+07
Evaluation/AverageDiscountedReturn   

KeyboardInterrupt: 